# CMCL Recommender System - GPU Run (Goodreads Poetry)

This notebook is designed to train the Cross-Modal Contrastive Learning (CMCL) model on the Goodreads Poetry dataset using a GPU. It is optimized for Google Colab **FREE TIER** (12.7 GB RAM limit).

## 1. Setup Codebase
Cloning the repository from GitHub. If it already exists, we pull the latest changes.

In [ ]:
import os
if os.path.exists("PGMS_for_Recommender_Systems"):
    %cd PGMS_for_Recommender_Systems
    !git pull
else:
    !git clone https://github.com/mohdfaour03/PGMS_for_Recommender_Systems.git
    %cd PGMS_for_Recommender_Systems
!pip install -e .

## 2. Setup and Dependencies
Install necessary packages.

In [ ]:
!pip install torch pandas numpy scipy scikit-learn tqdm pyyaml requests

## 3. Imports and Configuration

In [ ]:
import sys
from pathlib import Path
import json
import torch
import gc

# Ensure the current directory is in the path so we can import coldstart
sys.path.append(".")

try:
    from coldstart.src import pipeline
    from coldstart.src.notebook_utils import build_goodreads_interaction_frame, _read_simple_yaml
except ImportError:
    print("❌ ERROR: Could not import 'coldstart'. Please ensure the repository was cloned successfully.")
    raise

# Check for GPU
print(f"GPU Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"Device: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
else:
    print("⚠️ WARNING: GPU not detected. Go to Runtime > Change runtime type > T4 GPU to enable it.")

## 4. Data Loading
Load the Goodreads Poetry dataset. **Using 100k interactions to fit within Colab free tier RAM limit (12.7 GB).**

In [ ]:
DATA_DIR = Path("coldstart/data")
DATA_DIR.mkdir(parents=True, exist_ok=True)
GOODREADS_PATH = DATA_DIR / "goodreads_poetry_100k.csv"
RUN_DIR = Path("coldstart/output/goodreads_gpu_run")
RUN_DIR.mkdir(parents=True, exist_ok=True)

# CRITICAL: Using 100k interactions instead of 300k to avoid RAM crash
INTERACTION_LIMIT = 100000

if not GOODREADS_PATH.exists():
    print(f"Downloading/Loading Goodreads Poetry dataset ({INTERACTION_LIMIT} interactions)...")
    df = build_goodreads_interaction_frame(genre="poetry", limit=INTERACTION_LIMIT)
    df.to_csv(GOODREADS_PATH, index=False)
    print(f"Saved Goodreads Poetry dataset to {GOODREADS_PATH}")
else:
    print(f"Goodreads Poetry dataset already exists at {GOODREADS_PATH}")

gc.collect()

## 5. Pipeline Execution
Run the training pipeline. We enable `prefer_gpu=True` and use conservative batch sizes for RAM safety.

In [ ]:
# Load base config
config = _read_simple_yaml("coldstart/configs/base.yaml")

# Prepare Dataset
print("Preparing dataset...")
pipeline.prepare_dataset(
    GOODREADS_PATH,
    RUN_DIR,
    tfidf_params=config.get("tfidf", {}),
    cold_item_frac=0.2,
    seed=42,
    interaction_limit=INTERACTION_LIMIT,
)

gc.collect()

# Train and Evaluate
print("Starting training on GPU...")
metrics = pipeline.train_and_evaluate_content_model(
    RUN_DIR,
    k_factors=16,
    k_eval=5,
    mf_reg=float(config.get("mf", {}).get("reg", 0.02)),
    mf_iters=20, # Reduced from 30 for speed
    mf_lr=float(config.get("mf", {}).get("lr", 0.02)),
    seed=42,
    ctrlite_reg=float(config.get("ctrlite", {}).get("reg", 0.01)),
    ctrlite_lr=float(config.get("ctrlite", {}).get("lr", 0.1)),
    ctrlite_iters=50, # Reduced from 80 for speed
    adaptive=False,
    model="ctrlite,cdl,cmcl", # Run all for comparison
    a2f_cfg=config.get("a2f", {}),
    ctpf_cfg=config.get("ctpf", {}),
    cdl_cfg=config.get("cdl", {}),
    hft_cfg=config.get("hft", {}),
    micm_cfg=config.get("micm", {}),
    cmcl_cfg={"iters": 10},
    backend="torch",
    prefer_gpu=True, # ENABLE GPU
    mf_cfg={
        "batch_size": 1024,
        "score_batch_size": 2048,
        "infer_batch_size": 2048,
        "ctrlite_batch_size": 1024,
    },
)

gc.collect()
print("\n✅ Training completed successfully!")

## 6. Results

In [ ]:
print("\n=== Final Results ===")
print(json.dumps(metrics, indent=2))
with open(RUN_DIR / "metrics.json", "w") as f:
    json.dump(metrics, f, indent=2)

print(f"\nResults saved to {RUN_DIR / 'metrics.json'}")